In [4]:
import pandas as pd

def carregar_dados(caminho_csv):
    """Carrega o CSV e converte timestamp."""
    df = pd.read_csv(caminho_csv)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

def gerar_features(df):
    """Cria todas as features para análise."""
    # Features temporais
    df['data'] = df['timestamp'].dt.date
    df['hora'] = df['timestamp'].dt.hour
    df['minuto'] = df['timestamp'].dt.minute
    df['dia_da_semana'] = df['timestamp'].dt.day_name()
    df['final_de_semana'] = df['dia_da_semana'].isin(['Saturday', 'Sunday']).astype(int)

    # Variação e tendência
    df['var_jogadores'] = df.groupby('ip')['playerCount'].diff()
    df['pct_var_jogadores'] = df.groupby('ip')['playerCount'].pct_change() * 100
    df['media_movel_10'] = df.groupby('ip')['playerCount'].transform(lambda x: x.rolling(window=10, min_periods=1).mean())
    df['media_movel_30'] = df.groupby('ip')['playerCount'].transform(lambda x: x.rolling(window=30, min_periods=1).mean())
    df['desvio_movel_30'] = df.groupby('ip')['playerCount'].transform(lambda x: x.rolling(window=30, min_periods=1).std())

    # Popularidade relativa
    df['total_jogadores'] = df.groupby('timestamp')['playerCount'].transform('sum')
    df['proporcao_rede'] = df['playerCount'] / df['total_jogadores']

    # Flags de eventos
    limite_pico = df['playerCount'].quantile(0.95)
    df['flag_pico'] = (df['playerCount'] > limite_pico).astype(int)
    df['queda_abrupta'] = (df['pct_var_jogadores'] < -20).astype(int)
    df['recuperacao'] = (df['pct_var_jogadores'] > 20).astype(int)

    # Ciclos de demanda
    df['periodo_dia'] = pd.cut(
        df['hora'],
        bins=[0, 6, 12, 18, 24],
        labels=['Madrugada', 'Manhã', 'Tarde', 'Noite'],
        right=False
    )

    # Intervalos entre registros
    df['intervalo_segundos'] = df.groupby('ip')['timestamp'].diff().dt.total_seconds()

    # Codificação para ML
    df['server_id'] = df['ip'].astype('category').cat.codes
    df['servidor_hora'] = df['ip'] + "_" + df['hora'].astype(str)

    return df

def salvar_dados(df, caminho_saida):
    """Salva o dataset enriquecido em CSV."""
    df.to_csv(caminho_saida, index=False)
    print(f"Dataset salvo em: {caminho_saida}")

def main():
    entrada = "https://dl.minetrack.me/Java/1-8-2021.csv"  # caminho do CSV original
    saida = "minecraft_servidores_features.csv"

    print("Carregando dados...")
    df = carregar_dados(entrada)

    print("Gerando features...")
    df = gerar_features(df)

    print("Salvando dataset enriquecido...")
    salvar_dados(df, saida)

    print("Processo concluído!")

if __name__ == "__main__":
    main()


Carregando dados...
Gerando features...


C:\Users\user\AppData\Local\Temp\ipykernel_14044\3348501601.py:20: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['pct_var_jogadores'] = df.groupby('ip')['playerCount'].pct_change() * 100


Salvando dataset enriquecido...
Dataset salvo em: minecraft_servidores_features.csv
Processo concluído!


In [11]:
!dir

 O volume na unidade Z n�o tem nome.
 O N�mero de S�rie do Volume � 0000-0000

 Pasta de Z:\home\ubuntu\Projeto\stack-mine-track\mine-tracker\notebooks

28/08/2025  13:14        57.878.539 minecraft_servidores_features.csv
28/08/2025  13:07                50 Mine_FitModel.ipynb?Zone.Identifier
28/08/2025  13:13            24.547 Mine_FeatureEngineer.ipynb
28/08/2025  13:10           234.412 Mine_FitModel.ipynb
28/08/2025  13:07                50 Mine_FeatureEngineer.ipynb?Zone.Identifier
28/08/2025  13:13    <DIR>          .
28/08/2025  13:07                 0 .gitkeep
28/08/2025  13:13    <DIR>          ..
               6 arquivo(s)     58.137.598 bytes
               2 pasta(s)   992.212.377.600 bytes dispon�veis


In [3]:
!pip install pandas numpy matplotlib seaborn scikit-learn  -q 

In [12]:
# -*- coding: utf-8 -*-
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from textwrap import fill

DATA_PATH = "minecraft_servidores_features.csv"
OUTPUT_DIR = "report_segmentado"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === Configurações de segmentação ===
# method = "P95"  -> usa percentil 95 global
# method = "IQR"  -> usa Q3 + 1.5*IQR global
# method = "LIST" -> trata uma lista de servidores como outliers (ex.: Hypixel)
METHOD = "P95"
OUTLIER_SERVERS = {"mc.hypixel.net"}   # usado se METHOD == "LIST"

# === Helpers ===
def annotate_figure(fig, title, subtitle, description, actions):
    fig.suptitle(title, fontsize=14, fontweight="bold", y=0.98)
    if subtitle:
        fig.text(0.5, 0.94, subtitle, ha="center", va="top")
    fig.text(0.01, -0.02, fill("Descrição: " + description, width=110), ha="left", va="top")
    fig.text(0.01, -0.08, fill("Ações sugeridas: " + actions, width=110), ha="left", va="top")
    fig.tight_layout(rect=[0, 0.1, 1, 0.92])

def save_fig(fig, name):
    path = os.path.join(OUTPUT_DIR, name)
    fig.savefig(path, bbox_inches="tight")
    plt.close(fig)
    return path

def ensure_time_cols(df):
    if 'timestamp' in df.columns and np.issubdtype(df['timestamp'].dtype, np.number):
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms', errors='coerce')
    if 'hora' not in df.columns and 'timestamp' in df.columns:
        df['hora'] = df['timestamp'].dt.hour
    if 'dia_da_semana' not in df.columns and 'timestamp' in df.columns:
        df['dia_da_semana'] = df['timestamp'].dt.day_name()
    ordem_dias = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    if 'dia_da_semana' in df.columns:
        try:
            df['dia_da_semana'] = pd.Categorical(df['dia_da_semana'], categories=ordem_dias, ordered=True)
        except Exception:
            pass
    return df

def segmentar(df, method="P95"):
    pc = df['playerCount']
    if method.upper() == "P95":
        thr = pc.quantile(0.95)
        mask_core = pc <= thr
        criterio = f"P95 global = {thr:.0f}"
    elif method.upper() == "IQR":
        q1, q3 = pc.quantile([0.25, 0.75])
        iqr = q3 - q1
        thr = q3 + 1.5 * iqr
        mask_core = pc <= thr
        criterio = f"IQR global: Q3 + 1.5*IQR = {thr:.0f}"
    elif method.upper() == "LIST":
        mask_core = ~df['ip'].isin(OUTLIER_SERVERS)
        criterio = f"LISTA de servidores outliers: {sorted(OUTLIER_SERVERS)}"
        thr = None
    else:
        raise ValueError("METHOD inválido. Use 'P95', 'IQR' ou 'LIST'.")
    return df[mask_core].copy(), df[~mask_core].copy(), criterio, thr

def hist_playercount(df, tag, same_xlim=None, show_log=False, thr_line=None):
    fig = plt.figure(figsize=(10,5))
    ax = fig.add_subplot(111)
    ax.hist(df['playerCount'].dropna(), bins=50, alpha=0.9, label="playerCount", log=show_log)
    ax.set_xlabel("Número de jogadores")
    ax.set_ylabel("Frequência (registros)" + (" (log)" if show_log else ""))
    media_pc = df['playerCount'].mean()
    mediana_pc = df['playerCount'].median()
    ax.axvline(media_pc, linestyle="--", linewidth=1, label="Média")
    ax.axvline(mediana_pc, linestyle="--", linewidth=1, label="Mediana")
    if thr_line is not None:
        ax.axvline(thr_line, linestyle="--", linewidth=1, label="Limiar")
    if same_xlim is not None:
        ax.set_xlim(0, same_xlim)
    ax.legend(loc="upper right")
    desc = (f"Distribuição de playerCount ({tag}). Média ≈ {int(media_pc)}, mediana ≈ {int(mediana_pc)}. "
            "Sem outliers, a forma reflete melhor o comportamento típico.")
    acoes = ("Usar esse recorte para definir SLAs e alertas realistas; "
             "comparar com o segmento de outliers para entender picos excepcionais.")
    annotate_figure(fig, f"Histograma – {tag}", "Média e mediana destacadas", desc, acoes)
    return save_fig(fig, f"01_hist_{tag}.png")

def boxplot_top(df, tag, top_n=10, ylim=None):
    top_ips = df['ip'].value_counts().head(top_n).index.tolist()
    df_top = df[df['ip'].isin(top_ips)]
    medianas = df_top.groupby('ip')['playerCount'].median().sort_values(ascending=False)
    ordered_ips = medianas.index.tolist()
    data_box = [df_top[df_top['ip']==ip]['playerCount'].dropna() for ip in ordered_ips]
    fig = plt.figure(figsize=(11,6))
    ax = fig.add_subplot(111)
    ax.boxplot(data_box, labels=ordered_ips, vert=True, showfliers=False)
    ax.set_xlabel(f"Servidores (Top {top_n} por volume) – {tag}")
    ax.set_ylabel("Número de jogadores")
    ax.tick_params(axis='x', rotation=30)
    if ylim is not None:
        ax.set_ylim(0, ylim)
    desc = ("Comparação de distribuição por servidor. Caixas altas/estreitas = popularidade/estabilidade; "
            "caixas baixas/largas = menor uso/alta variância.")
    acoes = ("Focar otimizações em servidores com mediana alta; investigar variância alta caso a caso.")
    annotate_figure(fig, f"Boxplot por servidor – {tag}",
                    f"Top {top_n} servidores (sem outliers desse segmento)", desc, acoes)
    return save_fig(fig, f"02_boxplot_{tag}.png")

def mean_by_hour_plot(df, tag, xlim=None, ylim=None):
    mean_by_hour = df.groupby('hora', dropna=True)['playerCount'].mean()
    hora_pico = int(mean_by_hour.idxmax())
    valor_pico = float(mean_by_hour.max())
    fig = plt.figure(figsize=(10,5))
    ax = fig.add_subplot(111)
    ax.bar(mean_by_hour.index.astype(int), mean_by_hour.values, label="Média por hora")
    ax.set_xlabel("Hora do dia")
    ax.set_ylabel("Média de jogadores")
    ax.set_xticks(range(0,24,1))
    ax.axvline(hora_pico, linestyle="--", linewidth=1, label="Hora de pico")
    if ylim is not None:
        ax.set_ylim(0, ylim)
    ax.legend(loc="upper right")
    desc = (f"Pico ~{hora_pico}h (≈{int(valor_pico)}). Segmentar evita que poucos valores extremos "
            "mascarem a curva diária típica.")
    acoes = ("Agendar manutenção fora do pico; alocar recursos de acordo com a curva do segmento.")
    annotate_figure(fig, f"Média por hora – {tag}",
                    "Padrões circadianos do segmento", desc, acoes)
    return save_fig(fig, f"03_media_hora_{tag}.png"), hora_pico, valor_pico

def weekday_plot(df, tag, ylim=None):
    mean_by_weekday = df.groupby('dia_da_semana', dropna=True)['playerCount'].mean()
    if len(mean_by_weekday)>0:
        dia_pico = str(mean_by_weekday.idxmax())
        val_pico = float(mean_by_weekday.max())
    else:
        dia_pico, val_pico = "N/A", 0.0
    fig = plt.figure(figsize=(10,5))
    ax = fig.add_subplot(111)
    ax.bar(range(len(mean_by_weekday.index)), mean_by_weekday.values, label="Média por dia")
    ax.set_xlabel("Dia da semana")
    ax.set_ylabel("Média de jogadores")
    ax.set_xticks(range(len(mean_by_weekday.index)))
    ax.set_xticklabels([str(x) for x in mean_by_weekday.index], rotation=20)
    if ylim is not None:
        ax.set_ylim(0, ylim)
    ax.legend(loc="upper right")
    desc = (f"{tag}: maior média em {dia_pico} (≈{int(val_pico)}). Sem outliers, diferenças semanais emergem com clareza.")
    acoes = ("Planejar campanhas/updates para dias fortes; comparar com segmento de outliers.")
    annotate_figure(fig, f"Média por dia da semana – {tag}",
                    "Variação semanal sem viés de extremos", desc, acoes)
    return save_fig(fig, f"04_media_semana_{tag}.png")

def heatmap_hour_day(df, tag):
    pivot = df.pivot_table(values='playerCount', index='dia_da_semana', columns='hora', aggfunc='mean')
    fig = plt.figure(figsize=(12,6))
    ax = fig.add_subplot(111)
    im = ax.imshow(pivot.values, aspect='auto')
    ax.set_xlabel("Hora do dia"); ax.set_ylabel("Dia da semana")
    ax.set_xticks(range(pivot.shape[1])); ax.set_xticklabels(list(pivot.columns))
    ax.set_yticks(range(pivot.shape[0])); ax.set_yticklabels([str(x) for x in pivot.index])
    cbar = fig.colorbar(im, ax=ax); cbar.set_label("Média de jogadores")
    desc = ("Mapa hora×dia do segmento. Sem extremos, blocos de alta/baixa intensidade ficam nítidos.")
    acoes = ("Autoscaling por faixas; jobs pesados nas janelas frias do segmento.")
    annotate_figure(fig, f"Heatmap hora × dia – {tag}",
                    "Média de jogadores por hora em cada dia", desc, acoes)
    return save_fig(fig, f"05_heatmap_{tag}.png")

def correlation_map(df, tag):
    cols = [c for c in ['playerCount','hora','final_de_semana','media_movel_10',
                        'media_movel_30','desvio_movel_30','proporcao_rede','pct_var_jogadores']
            if c in df.columns]
    corr = df[cols].corr(numeric_only=True)
    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111)
    im = ax.imshow(corr.values, aspect='auto')
    ax.set_xticks(range(len(cols))); ax.set_xticklabels(cols, rotation=30, ha='right')
    ax.set_yticks(range(len(cols))); ax.set_yticklabels(cols)
    cbar = fig.colorbar(im, ax=ax); cbar.set_label("Correlação de Pearson")
    desc = ("Relações lineares dentro do segmento. Correlações diferentes entre segmentos indicam dinâmicas distintas.")
    acoes = ("Escolher features por segmento; treinar modelos separados se necessário.")
    annotate_figure(fig, f"Correlação – {tag}",
                    "Força das relações lineares (Pearson)", desc, acoes)
    return save_fig(fig, f"06_correlacao_{tag}.png")

def series_top_server(df, tag):
    servidor_top = df['ip'].value_counts().idxmax()
    serie = df[df['ip']==servidor_top].sort_values('timestamp' if 'timestamp' in df.columns else df.index)
    fig = plt.figure(figsize=(12,5))
    ax = fig.add_subplot(111)
    ax.plot(serie['timestamp'], serie['playerCount'], label=f"{servidor_top}")
    ax.set_xlabel("Tempo"); ax.set_ylabel("Jogadores"); ax.legend(loc="upper right")
    q95 = serie['playerCount'].quantile(0.95)
    ax.axhline(q95, linestyle="--", linewidth=1, label="P95 servidor"); ax.legend()
    desc = (f"Série do servidor mais popular no segmento ({servidor_top}). Linha P95 orienta alertas.")
    acoes = ("Alarmes > P95; inspeção de quedas bruscas; previsão por servidor.")
    annotate_figure(fig, f"Série temporal – {tag}",
                    f"Evolução do top servidor – {servidor_top}", desc, acoes)
    return save_fig(fig, f"07_serie_top_{tag}.png")

def main():
    df = pd.read_csv(DATA_PATH)
    df = ensure_time_cols(df)

    # Segmentação
    core, out, criterio, thr = segmentar(df, METHOD)

    # Guardar para rastreabilidade
    core.to_csv(os.path.join(OUTPUT_DIR, f"segment_core_{METHOD}.csv"), index=False)
    out.to_csv(os.path.join(OUTPUT_DIR, f"segment_outliers_{METHOD}.csv"), index=False)

    # Para comparação justa, travo os limites Y com base no core (onde vamos analisar “normalidade”)
    yref = np.nanpercentile(core['playerCount'], 99)  # robusto
    # Limite X para hist do core (visão sem extremos)
    xref = core['playerCount'].max()

    # ----- Gráficos CORE -----
    h1 = hist_playercount(core, f"CORE_{METHOD}", same_xlim=xref, thr_line=thr)
    b1 = boxplot_top(core, f"CORE_{METHOD}", ylim=yref)
    m1, hora_pico_core, pico_core = mean_by_hour_plot(core, f"CORE_{METHOD}", ylim=yref)
    w1 = weekday_plot(core, f"CORE_{METHOD}", ylim=yref)
    hm1 = heatmap_hour_day(core, f"CORE_{METHOD}")
    c1 = correlation_map(core, f"CORE_{METHOD}")
    s1 = series_top_server(core, f"CORE_{METHOD}")

    # ----- Gráficos OUTLIERS -----
    # Para não enviesar, mantemos a mesma estrutura, sem travar eixos do outliers (pode ser muito maior)
    h2 = hist_playercount(out, f"OUTLIERS_{METHOD}", same_xlim=None, thr_line=thr)
    b2 = boxplot_top(out, f"OUTLIERS_{METHOD}")
    m2, hora_pico_out, pico_out = mean_by_hour_plot(out, f"OUTLIERS_{METHOD}")
    w2 = weekday_plot(out, f"OUTLIERS_{METHOD}")
    hm2 = heatmap_hour_day(out, f"OUTLIERS_{METHOD}")
    c2 = correlation_map(out, f"OUTLIERS_{METHOD}")
    s2 = series_top_server(out, f"OUTLIERS_{METHOD}")

    # ----- Visão “com tudo” apenas para referência (log no hist) -----
    fig = plt.figure(figsize=(10,5))
    ax = fig.add_subplot(111)
    ax.hist(df['playerCount'].dropna(), bins=50, alpha=0.9, log=True)
    ax.set_xlabel("Número de jogadores"); ax.set_ylabel("Frequência (log)")
    annotate_figure(fig, "Histograma geral (log)", "Referência com todos os dados",
                    "Escala log para visualizar a cauda de valores muito altos.",
                    "Usar somente como referência global; análises operacionais devem usar o segmento CORE.")
    all_log = save_fig(fig, "00_hist_geral_log.png")

    # ----- Report curto -----
    report = os.path.join(OUTPUT_DIR, "README_segmentado.txt")
    with open(report, "w", encoding="utf-8") as f:
        f.write("RELATÓRIO SEGMENTADO – Evitando viés por outliers\n\n")
        f.write(f"Método de segmentação: {METHOD} ({criterio})\n")
        if thr is not None:
            f.write(f"Limiar numérico: {thr:.0f}\n")
        f.write(f"Tamanho CORE: {len(core):,} | OUTLIERS: {len(out):,}\n")
        f.write(f"Pico médio por hora (CORE): ~{pico_core:.0f} às {hora_pico_core}h\n")
        f.write(f"Pico médio por hora (OUTLIERS): ~{pico_out:.0f} às {hora_pico_out}h\n")
        f.write("\nArquivos gerados na pasta 'report_segmentado/'.\n")

    print("OK! Relatórios e gráficos salvos em:", os.path.abspath(OUTPUT_DIR))

if __name__ == "__main__":
    main()


C:\Users\user\AppData\Local\Temp\ipykernel_14044\2637369781.py:99: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(data_box, labels=ordered_ips, vert=True, showfliers=False)
C:\Users\user\AppData\Local\Temp\ipykernel_14044\2637369781.py:134: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_by_weekday = df.groupby('dia_da_semana', dropna=True)['playerCount'].mean()
C:\Users\user\AppData\Local\Temp\ipykernel_14044\2637369781.py:157: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = df.pivot_table(valu

OK! Relatórios e gráficos salvos em: \\wsl.localhost\Ubuntu\home\ubuntu\Projeto\stack-mine-track\mine-tracker\notebooks\report_segmentado


Total de outliers: 14032
                    ip  playerCount
233650  mc.hypixel.net     106945.0
233660  mc.hypixel.net     106943.0
232950  mc.hypixel.net     106943.0
233700  mc.hypixel.net     106935.0
233060  mc.hypixel.net     106933.0
233760  mc.hypixel.net     106933.0
233490  mc.hypixel.net     106932.0
233040  mc.hypixel.net     106930.0
233500  mc.hypixel.net     106930.0
233640  mc.hypixel.net     106930.0
